In [1]:
import pandas as pd
import numpy as np
from scipy.linalg import inv
from PortOpt_factor.optimizer import pyport
from ipca_utils import impute_w_median, normalize, IPCA_factor


#### Read Raw Data and Preprocess

In [2]:
fn = "data/kelly_data_without_nanocap.p"
df = pd.read_pickle(fn)
cols_to_drop = ["isin", "cusip", "sedol", "excntry"]
df = df.drop(cols_to_drop, axis=1)
df = df.dropna(subset=['ret_local_lead1m'])

characteristics = df.columns[6:] #list of characteristics
df_ipca = impute_w_median(df, characteristics)
df_ipca = normalize(df_ipca, characteristics)

pd.set_option("display.max_column", None)
df.head(10)

,id,eom,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,100100001,1970-09-30,0.059230,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.389696,0.333709,1.095773,NaN,0.077817,-0.613949,NaN,NaN,NaN,8.273810,4.052973,0.147433,1.399905,-0.425303,1.022609,0.525045,0.284973,16.828861,-0.021824,3.848362,2.787572,2.823370,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.083380,-0.214719,NaN,0.032843,0.251010,0.266091,7.349377,3.858437,NaN,NaN,0.251010,0.060123,NaN,0.052671,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.838230,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,0.335870,NaN,NaN,1.129423,NaN,NaN,117,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,26.202500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,11.750,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.388130,-0.166415,NaN,NaN,NaN,NaN,NaN,NaN
1,100100001,1970-10-31,-0.284600,-0.280000,0.063830,0.059230,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.366314,0.313686,1.030027,NaN,0.080072,-0.577112,NaN,NaN,NaN,8.273810,4.052973,0.140456,1.288395,-0.399785,0.961253,0.500198,0.267874,16.979129,-0.021824,3.848362,2.787572,2.823370,2.191658,0.138015,1.048487,26.461111,40.889831,NaN,NaN,6.091646,10.403599,0.281903,0.452041,0.237322,0.312587,0.505851,0.193264,0.139454,0.222834,0.083380,-0.214719,NaN,0.032843,0.251010,0.266091,7.349377,3.858437,NaN,NaN,0.251010,NaN,NaN,NaN,NaN,0.222367,0.105129,2.115171,2.159265,0.425118,2.061128,0.434923,1.727343,0.172889,0.838230,-0.279152,-1.353428,6.0,-1.548939,NaN,2.143262,2.81187,-0.071796,NaN,NaN,1.129423,NaN,NaN,118,-7.392944,-5.827022,0.736966,-0.389403,NaN,0.626638,NaN,NaN,0.044476,-0.604826,0.060335,0.222692,3.209009,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,27.875000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12.500,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.063830,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.179708,-0.388130,-0.166415,0.557225,0.007569,-0.131257,-0.242119,1.131001,-1.038514
2,100100001,1970-11-30,0.106911,0.111111,-0.280000,-0.284600,NaN,NaN,NaN,NaN,0.055691,NaN,NaN,NaN,NaN,0.508769,0.435675,1.430593,NaN,0.111211,-0.801545,NaN,NaN,NaN,8.273810,4.052973,0.180267,1.789437,-0.555257,1.335073,0.641973,0.3

#### Read Preprocessed Data

In [6]:
df_ipca = pd.read_csv('data/factor_data_qnormed.csv')
characteristics = df_ipca.columns[6:] #list of characteristics
pd.set_option("display.max_column", None)
df_ipca.head(10)

,eom,id,ret_exc_lead1m,ret_local_lead1m,ret_local,ret_exc,niq_su,ret_6_1,ret_12_1,saleq_su,tax_gr1a,ni_inc8q,prc_highprc_252d,resff3_6_1,resff3_12_1,be_me,debt_me,at_me,ret_60_12,ni_me,fcf_me,div12m_me,eqpo_me,eqnpo_me,sale_gr3,sale_gr1,ebitda_mev,sale_me,ocf_me,ival_me,bev_mev,netdebt_me,eq_dur,capex_abn,at_gr1,ppeinv_gr1a,noa_at,noa_gr1a,lnoa_gr1a,capx_gr1,capx_gr2,capx_gr3,chcsho_12m,eqnpo_12m,debt_gr3,inv_gr1,inv_gr1a,oaccruals_at,taccruals_at,cowc_gr1a,coa_gr1a,col_gr1a,nncoa_gr1a,ncoa_gr1a,ncol_gr1a,nfna_gr1a,sti_gr1a,lti_gr1a,fnl_gr1a,be_gr1a,oaccruals_ni,taccruals_ni,netis_at,eqnetis_at,dbnetis_at,niq_be,niq_be_chg1,niq_at,niq_at_chg1,ebit_bev,ebit_sale,sale_bev,at_turnover,gp_at,gp_atl1,ope_be,ope_bel1,op_at,op_atl1,cop_at,cop_atl1,f_score,o_score,z_score,pi_nix,at_be,saleq_gr1,rd_me,rd_sale,opex_at,emp_gr1,rd5_at,age,dsale_dinv,dsale_drec,dgp_dsale,dsale_dsga,sale_emp_gr1,tangibility,kz_index,ocfq_saleq_std,cash_at,ni_ar1,ni_ivol,earnings_variability,aliq_at,aliq_mat,seas_1_1an,seas_1_1na,seas_2_5an,seas_2_5na,seas_6_10an,seas_6_10na,seas_11_15an,seas_11_15na,seas_16_20an,seas_16_20na,market_equity,ivol_ff3_21d,ivol_capm_252d,ivol_capm_21d,ivol_hxz4_21d,rvol_21d,beta_60m,betabab_1260d,beta_dimson_21d,turnover_126d,turnover_var_126d,dolvol_126d,dolvol_var_126d,prc,ami_126d,zero_trades_21d,zero_trades_126d,zero_trades_252d,rmax1_21d,rskew_21d,iskew_capm_21d,iskew_ff3_21d,iskew_hxz4_21d,coskew_21d,ret_1_0,betadown_252d,bidaskhl_21d,ret_3_1,ret_9_1,ret_12_7,corr_1260d,rmax5_21d,rmax5_rvol_21d,ni_be,ocf_at,ocf_at_chg1,mispricing_perf,mispricing_mgmt,qmj,qmj_prof,qmj_growth,qmj_safety
0,1962-01-31,100104301,-0.039478,-0.037478,NaN,NaN,NaN,NaN,NaN,NaN,0.276667,NaN,NaN,NaN,NaN,0.076667,0.156667,0.050000,NaN,0.113333,0.653333,NaN,NaN,NaN,0.623333,0.420000,0.106667,0.080000,0.333333,0.060000,0.103333,0.343333,0.936667,0.336667,0.430000,0.626667,0.366667,0.703333,0.270000,0.170000,0.376667,0.740000,NaN,NaN,0.770000,0.656667,0.720000,0.703333,0.726667,0.773333,0.666667,0.233333,0.253333,0.293333,0.206667,0.333333,NaN,0.423333,0.676667,0.730000,0.353333,0.660000,NaN,NaN,0.680000,NaN,NaN,NaN,NaN,0.800000,0.776667,0.563333,0.570000,0.906667,0.886667,0.806667,0.783333,0.893333,0.883333,0.880000,0.866667,0.738333,0.140000,NaN,0.863333,0.170000,NaN,0.036667,0.073333,0.560000,0.773333,0.056667,1.000000,0.223333,0.396667,0.700000,0.643333,0.193333,0.650000,NaN,NaN,0.566667,0.103333,0.160000,0.210000,0.686667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.840000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.906667,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.783333,0.460000,0.840000,NaN,NaN,NaN,NaN,NaN,NaN
1,1962-01-31,100113301,0.051937,0.053937,NaN,NaN,NaN,NaN,NaN,NaN,0.053333,NaN,NaN,NaN,NaN,0.500000,0.536667,0.966667,NaN,0.020000,0.180000,NaN,NaN,NaN,0.256667,0.130000,0.016667,0.973333,0.360000,0.500000,0.980000,0.100000,0.500000,0.310000,0.050000,0.080000,0.500000,0.498333,0.093333,0.216667,0.310000,0.260000,NaN,NaN,0.030000,0.150000,0.130000,0.500000,0.498333,0.113333,0.040000,0.080000,0.498333,0.182906,0.498333,0.583333,NaN,0.783333,0.620000,0.498333,0.498333,0.498333,NaN,NaN,0.613333,NaN,NaN,NaN,NaN,0.016667,0.023333,0.820000,0.726667,0.333333,0.270000,0.500000,0.500000,0.016667,0.016667,0.500000,0.498333,0.161667,0.500000,NaN,0.500000,0.500000,NaN,0.498333,0.498333,0.840000,0.203333,0.500000,1.000000,0.776667,0.803333,0.143333,0.073333,0.220000,0.130000,NaN,NaN,0.403333,0.030000,0.786667,0.906667,0.190000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.133333,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.060000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.500000,0.310000,0.246667,NaN,NaN,NaN,NaN,NaN,NaN
2,1962-01-31,100115701,0.062263,0.064263,NaN,NaN,NaN,NaN,NaN,NaN,0.796667,NaN,NaN,NaN,NaN,0.500000,0.000000,0.266667,NaN,0.150000,0.770000,NaN,NaN,NaN,0.963333,0.806667,0.226667,0.606667,0.660000,0.500000,0.133333,0.336667,0.500000,0.103333,0.930000,0.650000,0

In [7]:
pd.set_option('display.max_columns', None)
df_ipca.columns[df_ipca.isna().any(axis=0)]

Index(['ret_local', 'ret_exc', 'niq_su', 'ret_6_1', 'ret_12_1', 'saleq_su',
       'ni_inc8q', 'prc_highprc_252d', 'resff3_6_1', 'resff3_12_1',
       'ret_60_12', 'div12m_me', 'eqpo_me', 'eqnpo_me', 'chcsho_12m',
       'eqnpo_12m', 'sti_gr1a', 'netis_at', 'eqnetis_at', 'niq_be',
       'niq_be_chg1', 'niq_at', 'niq_at_chg1', 'z_score', 'saleq_gr1',
       'kz_index', 'ocfq_saleq_std', 'aliq_mat', 'seas_1_1an', 'seas_1_1na',
       'seas_2_5an', 'seas_2_5na', 'seas_6_10an', 'seas_6_10na',
       'seas_11_15an', 'seas_11_15na', 'seas_16_20an', 'seas_16_20na',
       'ivol_ff3_21d', 'ivol_capm_252d', 'ivol_capm_21d', 'ivol_hxz4_21d',
       'rvol_21d', 'beta_60m', 'betabab_1260d', 'beta_dimson_21d',
       'turnover_126d', 'turnover_var_126d', 'dolvol_126d', 'dolvol_var_126d',
       'ami_126d', 'zero_trades_21d', 'zero_trades_126d', 'zero_trades_252d',
       'rmax1_21d', 'rskew_21d', 'iskew_capm_21d', 'iskew_ff3_21d',
       'iskew_hxz4_21d', 'coskew_21d', 'ret_1_0', 'betadown_252d',


## IPCA Parameters

In [8]:
from logger import ErrorLogger
from tqdm import tqdm
import os
from datetime import datetime

In [12]:
window_size = 240 #rolling window size
K = 6 #num of principle components
unique_dates = sorted(df_ipca['eom'].unique()) #unique dates
T = len(unique_dates)

current_date = datetime.now().strftime('%Y-%m-%d')
# log_fp = "logs/"+f"{current_date}-w{window_size}-log-error.txt"
res_fp = "results/"+f"{current_date}-w{window_size}-results.csv"
logger = ErrorLogger() #each node/window has separate log file

print (f'There are {T} unique days starting with {unique_dates[0]}')
print (f'Total {len(characteristics)} charateristics starting with {characteristics[0]}')

There are 745 unique days starting with 1962-01-31
Total 153 charateristics starting with niq_su


In [20]:
problem_date = '2003-10-31'
for t in range(window_size, T):
    if unique_dates[t] == problem_date:
        print (t)
        break  

501


In [21]:
np.random.seed(122)

t = 501
K = 6
    
window_dates = unique_dates[t-window_size:t]
window_data = df_ipca[df_ipca['eom'].isin(window_dates)]
date_to_predict = unique_dates[t]

print (f'======Progress: {t}: {date_to_predict}======')

# calculate ipca
try:
    Gamma, Factors, r_t, excess_r_t, X_last = IPCA_factor(window_data, characteristics, K)
except Exception as e:
    raise

# regularization
V_t = inv(Gamma.T @ X_last.T @ X_last @ Gamma) @ Gamma.T @ X_last.T
reg_mat = np.zeros_like(V_t)
reg_mat[:K, :K] = np.eye(K)*1e-03
V_t += reg_mat

Sigma_t= np.cov(Factors, rowvar = True)
mu_t = np.array(np.mean(Factors, axis=1))

# Grid search? grid search for regularization terms
g1 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))
g2 = np.exp(np.linspace( np.log(1e-6),np.log(5),10))

OptimalPortfolioWeights_t = pyport.portfolio_optimization(
    meanVec=np.array(mu_t),
    sigMat=np.array(Sigma_t),
    retTarget=0,
    longShort=0.2,
    maxAlloc=0.08, #
    lambda_l1=g1[0],
    lambda_l2=g2[0],
    riskfree=0,
    assetsOrder=None,
    maxShar=1,
    factor=np.array(V_t.T)[:,0:K+1],
    turnover=None,
    w_pre=None,
    individual=False,
    exposure_constrain=0,
    w_bench=None,
    factor_exposure_constrain=None,
    U_factor=None,
    general_linear_constrain=None,
    U_genlinear=0,
    w_general=None,
    TE_constrain=0,
    general_quad=0,
    Q_w=None,
    Q_b=None,
    Q_bench=None
)[0].reshape(-1, 1)

ret_t = (V_t @ r_t).T @ OptimalPortfolioWeights_t

w_individual = np.dot(np.array(V_t.T), OptimalPortfolioWeights_t).flatten()


print (f'======Max_return: {t}: {ret_t}======')      
           
    

======Progress: 501: 2003-10-31======


The panel dimensions are:
n_samples: 9480 , L: 153 , T: 200


[========================================================================] 100%


Step 1 - Aggregate Update: 1.7689269891583321
Step 2 - Aggregate Update: 0.7362171154124526
Step 3 - Aggregate Update: 0.10485672146576723
Step 4 - Aggregate Update: 0.05797464959768109
Step 5 - Aggregate Update: 0.03434275333468351
Step 6 - Aggregate Update: 0.021169651830932196
Step 7 - Aggregate Update: 0.016309124607663747
Step 8 - Aggregate Update: 0.011371894507619995
Step 9 - Aggregate Update: 0.007860502837405553
Step 10 - Aggregate Update: 0.005281148686989409
Step 11 - Aggregate Update: 0.003421125988372173
Step 12 - Aggregate Update: 0.00216047460628431
Step 13 - Aggregate Update: 0.0013377254301143449
Step 14 - Aggregate Update: 0.0008140115305529938
Step 15 - Aggregate Update: 0.00048662194636239775
Step 16 - Aggregate Update: 0.0002849508395175171
Step 17 - Aggregate Update: 0.00017086313264449982
Step 18 - Aggregate Update: 0.00012105044141078047
Step 19 - Aggregate Update: 8.608275799716625e-05
-- Convergence Reached --
======Max_return: 501: [0.08072745]======
